In [2]:
import scipy.io as sio
import pandas as pd
import numpy as np

corpus = 'MozillaCommonVoice-ab'

data_dict = sio.loadmat('STM_output/Survey/speech_params_MozillaCommonVoice/ab/common_voice_ab_19904203_Params.mat')
structure_dict = {field: data_dict['Params'][field][0] for field in data_dict['Params'].dtype.names}
df = pd.DataFrame(structure_dict)
df.drop(columns=['x_axis','y_axis'], inplace=True)
df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
df
df['corpus'] = corpus


In [3]:
df

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,corpus
0,data/speechCorp/MozillaCommonVoice/ab/clips/co...,common_voice_ab_19904203,Abkhaz,1,1,384000,48,8,0.957167,8,MozillaCommonVoice-ab


In [5]:
import csv
valid_df = pd.read_csv('/Users/andrewchang/NYU_research/MusicSpeech-STM/data/speechCorp/MozillaCommonVoice/ab/validated.tsv', sep='\t', quoting=csv.QUOTE_NONE)
valid_df

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,02ed89b702eabb9b988cbdea8b093f836b1bbebaba5835...,common_voice_ab_19946913.mp3,"Аллаҳ хаҵеи-ԥҳәыси иеилыхны, аҭыԥҳацәа роума и...",2,1,thirties,female,NaN,NaN,ab,NaN
1,044dd6405d36e86179776c2277aca7a30d6cfc1666bfc7...,common_voice_ab_20341606.mp3,"Уи Ибраҳим, Аллаҳ иваҟәылара аԥшьызгоз дреиуам...",2,0,NaN,NaN,NaN,NaN,ab,NaN
2,62e59ee2e19794a42adddad4a522325b4243d75d2b71a3...,common_voice_ab_29675146.mp3,хԥа,2,0,twenties,female,NaN,NaN,ab,Benchmark
3,637977539b1a6b6a75b55ae95e2061bad48fc46ea48007...,common_voice_ab_29512133.mp3,ԥшьба,3,0,teens,female,NaN,NaN,ab,Benchmark
4,20aaed1ec78252874dfa641488e93fbaa62cfc7c0ae89f...,common_voice_ab_27923256.mp3,жәба,2,1,NaN,NaN,NaN,NaN,ab,Benchmark
...,...,...,...,...,...,...,...,...,...,...,...
41977,68f6a8d9cdcc1f9b48b1690327761a26e7735653c0d481...,common_voice_ab_29651036.mp3,Сыҳәҳәар сҭахын.,2,0,fifties,female,NaN,NaN,ab,NaN
41978,68f6a8d9cdcc1f9b48b1690327761a26e7735653c0d481...,common_voice_ab_29651038.mp3,Саныҟам ак ахьыр ҳәа сшәоит адунеи.,3,0,fifties,female,NaN,NaN,ab,NaN
41979,68f6a8d9cdcc1f9b48b1690327761a26e7735653c0d481...,common_voice_ab_29651039.mp3,"Ааи, дҳақьымуп, аҳақьым.",2,0,fifties,female,NaN,NaN,ab,NaN
41980,68f6a8d9cdcc1f9b48b1690327761a26e7735653c0d481...,common_voice_ab_29651041.mp3,Нцәа дызмам адәы иқәихуазеи!,2,0,fifties,female,NaN,NaN,ab,NaN


In [10]:
df['filepath'][0]


'data/speechCorp/TAT-Vol2/condenser/TH_THM0018/0029-1.1-03.wav'

In [67]:
structure_dict = {field: data_dict['Params'][field][0] for field in data_dict['Params'].dtype.names}

In [68]:
structure_dict

{'filepath': array([array(['data/speechCorp/TAT-Vol2/condenser/TH_THM0018/0029-1.1-03.wav'],
              dtype='<U61')                                                     ],
       dtype=object),
 'filename': array([array(['0029-1.1-03'], dtype='<U11')], dtype=object),
 'LangOrInstru': array([array(['Taiwanese'], dtype='<U9')], dtype=object),
 'VoiOrNot': array([array([[1]], dtype=uint8)], dtype=object),
 'startPoint': array([array([[1]], dtype=uint8)], dtype=object),
 'endPoint': array([array([[128000]], dtype=int32)], dtype=object),
 'FSkhz': array([array([[16]], dtype=uint8)], dtype=object),
 'expdur': array([array([[8]], dtype=uint8)], dtype=object),
 'maxsil': array([array([[0.3506875]])], dtype=object),
 'totalLengCur': array([array([[8]], dtype=uint8)], dtype=object),
 'x_axis': array([array([[-62.5 , -62.25, -62.  , -61.75, -61.5 , -61.25, -61.  , -60.75,
                -60.5 , -60.25, -60.  , -59.75, -59.5 , -59.25, -59.  , -58.75,
                -58.5 , -58.25, -58.  , -5

In [69]:
import pandas as pd
df = pd.DataFrame(structure_dict)
df

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,x_axis,y_axis
0,[data/speechCorp/TAT-Vol2/condenser/TH_THM0018...,[0029-1.1-03],[Taiwanese],[[1]],[[1]],[[128000]],[[16]],[[8]],[[0.3506875]],[[8]],"[[-62.5, -62.25, -62.0, -61.75, -61.5, -61.25,...","[[-14.0, -13.813333333333334, -13.626666666666..."


In [71]:
import numpy as np
df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
df

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,x_axis,y_axis
0,data/speechCorp/TAT-Vol2/condenser/TH_THM0018/...,0029-1.1-03,Taiwanese,[1],[1],[128000],[16],[8],[0.3506875],[8],"[-62.5, -62.25, -62.0, -61.75, -61.5, -61.25, ...","[-14.0, -13.813333333333334, -13.6266666666666..."


In [7]:
param_dict = mat73.loadmat('/Users/andrewchang/NYU_research/MusicSpeech-STM/results/Survey/music_params_Albouy2020Science/English_S01_M01_Params.mat')


In [12]:
param_dict['Params']['y_axis'].shape

(150,)

In [25]:
import matlab.engine

ModuleNotFoundError: No module named 'matlab'